<a href="https://colab.research.google.com/github/crystal0523/SVHN-digit-detection/blob/main/infer_speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import time
import os
import csv
import json
import cv2
import argparse
import pandas as pd
from google.colab import files
files.upload()
!unzip retinanet.zip

In [38]:
!unzip retinanet.zip

Archive:  retinanet.zip
 extracting: retinanet/__init__.py   
   creating: retinanet/__pycache__/
  inflating: retinanet/__pycache__/__init__.cpython-36.pyc  
  inflating: retinanet/__pycache__/__init__.cpython-37.pyc  
  inflating: retinanet/__pycache__/anchors.cpython-36.pyc  
  inflating: retinanet/__pycache__/anchors.cpython-37.pyc  
  inflating: retinanet/__pycache__/coco_eval.cpython-36.pyc  
  inflating: retinanet/__pycache__/csv_eval.cpython-36.pyc  
  inflating: retinanet/__pycache__/dataloader.cpython-36.pyc  
  inflating: retinanet/__pycache__/dataloader.cpython-37.pyc  
  inflating: retinanet/__pycache__/losses.cpython-36.pyc  
  inflating: retinanet/__pycache__/losses.cpython-37.pyc  
  inflating: retinanet/__pycache__/model.cpython-36.pyc  
  inflating: retinanet/__pycache__/model.cpython-37.pyc  
  inflating: retinanet/__pycache__/utils.cpython-36.pyc  
  inflating: retinanet/__pycache__/utils.cpython-37.pyc  
  inflating: retinanet/anchors.py    
  inflating: retinanet/

In [37]:
import torch
import numpy as np
import time
import os
import csv
import json
import cv2
import argparse
import pandas as pd
from timeit import timeit 
import datetime
def load_classes(csv_reader):
    result = {}

    for line, row in enumerate(csv_reader):
        line += 1

        try:
            class_name, class_id = row
        except ValueError:
            raise(ValueError('line {}: format should be \'class_name,class_id\''.format(line)))
        class_id = int(class_id)

        if class_name in result:
            raise ValueError('line {}: duplicate class name: \'{}\''.format(line, class_name))
        result[class_name] = class_id
    return result

image_path='/content/drive/My Drive/73.png'
model_path='/content/drive/My Drive/csv_retinanet2_7.pt'
class_list='/content/drive/My Drive/class.csv'

with open(class_list, 'r') as f:
    classes = load_classes(csv.reader(f, delimiter=','))

labels = {}
for key, value in classes.items():
    labels[value] = key

model = torch.load(model_path)

if torch.cuda.is_available():
    model = model.cuda()

model.training = False
model.eval()

image = cv2.imread(image_path)
rows, cols, cns = image.shape

smallest_side = min(rows, cols)
# rescale the image so the smallest side is min_side
min_side = 608
max_side = 1024
scale = min_side / smallest_side
largest_side = max(rows, cols)

if largest_side * scale > max_side:
    scale = max_side / largest_side

# resize the image with the computed scale
image = cv2.resize(image, (int(round(cols * scale)), int(round((rows * scale)))))
rows, cols, cns = image.shape

pad_w = 32 - rows % 32
pad_h = 32 - cols % 32

new_image = np.zeros((rows + pad_w, cols + pad_h, cns)).astype(np.float32)
new_image[:rows, :cols, :] = image.astype(np.float32)
image = new_image.astype(np.float32)
image /= 255
image -= [0.485, 0.456, 0.406]
image /= [0.229, 0.224, 0.225]
image = np.expand_dims(image, 0)
image = np.transpose(image, (0, 3, 1, 2))

with torch.no_grad():
  image = torch.from_numpy(image)

  if torch.cuda.is_available():
    image = image.cuda()


  def infer(image,model):
    scores, classification, transformed_anchors = model(image.cuda().float())


  infer(image,model)
  %timeit infer(image,model)

10 loops, best of 3: 93.7 ms per loop


In [13]:
infer('/content/drive/My Drive/73.png')
print('Testing inference time')
%timeit infer(image)

NameError: ignored

In [22]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 40547040S_黃敏涓_project1.cpp
 40547040S_黃敏涓_project1.exe
 73.png
'Anyfile Notepad Files'
 class.csv
'Colab Notebooks'
'Copy of group30_DLP2020_video.mp4'
'Copy of 經濟學.docx'
'Copy of 經濟學.gdoc'
 csv_retinanet2_7.pt
 CV_黃敏涓.pdf
 Desktop.rar
'Enhancing a Pairs Trading strategy with the application_implement.pptx'
'Enhancing a Pairs Trading strategy with the application.pptx'
'Generative adversarial network based telecom fraud detection at recieving bank.pptx'
 gesture_recognition
 GloVe.docx
'Google 相簿'
 group30_DLP2020_intro.mp4
 L1-Introduction-2018.pdf
'L5-MachineLearningBasics-2018 (v2).pdf'
 MSP微軟學生大使.pptx
'National Taiwan Normal University - Gongguan Campus 30.m4a'
 交大
 報到單.pdf
'手機APP 相關'
 新生報到表格-論文指導教授初步確認單.doc
 穿戴式手套之手勢辨識應用_期末報告.pdf
 經濟學.docx
